# *Car Price Prediction - Linear Regression...!


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer

In [3]:
# Loading Datas
train_df = pd.read_csv('train.csv')
index_col = 'id'
test_df = pd.read_csv('test.csv')

In [4]:
print("train data shape", train_df.shape)
print("test data shape", test_df.shape)

train data shape (188533, 13)
test data shape (125690, 12)


In [5]:
df.rename(columns={df.columns[1] : "Brand"})


NameError: name 'df' is not defined

In [ ]:

missing_values = train_df.isnull().sum()/train_df.shape[0] * 100
print("\nTrain Data Missing Values:\n\n", missing_values)

# Filter only columns with missing values
missing_values = missing_values[missing_values > 0]

# Plotting the missing values
plt.figure(figsize=(10, 6))
sns.barplot(x=missing_values.index, y=missing_values.values, palette="viridis", hue=missing_values.index, legend=False)
plt.xticks(rotation=90)
plt.title('Percentage of Missing Values per Column', fontsize=16)
plt.xlabel('Features', fontsize=12)
plt.ylabel('Percentage (%)', fontsize=12)
plt.show()

In [ ]:
missing_values = test_df.isnull().sum()/test_df.shape[0] * 100
print("\nTest Data Missing Values:\n\n", missing_values)

# Filter only columns with missing values
missing_values = missing_values[missing_values > 0]

# Plotting the missing values
plt.figure(figsize=(10, 6))
sns.barplot(x=missing_values.index, y=missing_values.values, palette="viridis")

plt.xticks(rotation=90)
plt.title('Percentage of Missing Values per Column', fontsize=16)
plt.xlabel('Features', fontsize=12)
plt.ylabel('Percentage (%)', fontsize=12)
plt.show()

In [ ]:
# 
#Fuel Type Column Missing Value
train_df['fuel_type'] = train_df['fuel_type'].fillna('Electricity')
test_df['fuel_type'] = test_df['fuel_type'].fillna('Electricity')

In [ ]:
train_df.head()

In [ ]:
#Title Column Missing Value
train_df["clean_title"] = train_df['clean_title'].fillna('No')
test_df["clean_title"] = test_df['clean_title'].fillna('No')


In [ ]:
train_df["accident"] = train_df['accident'].fillna('non reported')

test_df["accident"] = test_df['accident'].fillna('non reported')

In [ ]:
import re
def extract_engine_features(df):
    # Extract Horsepower
    df['Horsepower'] = df['engine'].apply(lambda x: float(re.search(r'(\d+(\.\d+)?)HP', x).group(1)) if re.search(r'(\d+(\.\d+)?)HP', x) else None)

    # Extract Displacement
    df['Displacement'] = df['engine'].apply(lambda x: float(re.search(r'(\d+\.\d+)L|(\d+\.\d+) Liter', x).group(1) or re.search(r'(\d+\.\d+)L|(\d+\.\d+) Liter', x).group(2)) if re.search(r'(\d+\.\d+)L|(\d+\.\d+) Liter', x) else None)

    # Extract Engine Type
    df['Engine Type'] = df['engine'].apply(lambda x: re.search(r'(V\d+|I\d+|Flat \d+|Straight \d+)', x).group(1) if re.search(r'(V\d+|I\d+|Flat \d+|Straight \d+)', x) else None)

    # Extract Cylinder Count
    df['Cylinder Count'] = df['engine'].apply(lambda x: int(re.search(r'(\d+) Cylinder', x).group(1)) if re.search(r'(\d+) Cylinder', x) else None)

    # Extract Fuel Type
    fuel_types = ['Gasoline', 'Diesel', 'Electric', 'Hybrid', 'Flex Fuel']
    df['Fuel Type'] = df['engine'].apply(lambda x: next((fuel for fuel in fuel_types if fuel in x), None))
    
    return df

# Applying the function to both train and test datasets
train_df = extract_engine_features(train_df)
test_df = extract_engine_features(test_df)

In [ ]:

#Extracting Model Age from Model year column¶
import datetime
current_year = datetime.datetime.now().year
train_df['model_age'] = current_year - train_df['model_year']
test_df['model_age'] = current_year - test_df['model_year']
train_df.head()


In [ ]:
# Drop unnecessary Columns
columns_to_drop = ['engine', 'model','model_year', 'Engine Type', 'Fuel Type']
train_df.drop(columns=[col for col in columns_to_drop if col in train_df])
test_df.drop(columns=[col for col in columns_to_drop if col in test_df])

In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
#Handling Missing values for new columns
# Iterative Imputer: Limit max iterations and increase tolerance
imputer = IterativeImputer(max_iter=10, tol=1e-3, random_state=0)

# Apply only to relevant columns that require more sophisticated imputation
train_df[['Horsepower', 'Displacement']] = imputer.fit_transform(train_df[['Horsepower', 'Displacement']])
test_df[['Horsepower', 'Displacement']] = imputer.transform(test_df[['Horsepower', 'Displacement']])

# Simple Imputer for less complex columns
simple_imputer = SimpleImputer(strategy='mean')
train_df['Cylinder Count'] = simple_imputer.fit_transform(train_df[['Cylinder Count']])
test_df['Cylinder Count'] = simple_imputer.transform(test_df[['Cylinder Count']])

In [ ]:
# transmission 

def map_transmission(transmission):
    # Standardize the input
    transmission = transmission.strip().lower()
    
    if any(kw in transmission for kw in ['a/t', 'automatic']):
        return 'Automatic'
    elif any(kw in transmission for kw in ['m/t', 'manual']):
        return 'Manual'
    elif any(kw in transmission for kw in ['cvt', 'variator']):
        return 'Variator'
    elif any(kw in transmission for kw in ['tiptronic']):
        return 'Tiptronic'
    else:
        return 'Other'

# Apply the function to the DataFrame
train_df['transmission'] = train_df['transmission'].apply(map_transmission)
test_df['transmission'] = test_df['transmission'].apply(map_transmission)

print(train_df['transmission'].value_counts())
print('='*20)
print(test_df['transmission'].value_counts())

In [ ]:
#
#Mapping Accident,Clean Title and Transmission column¶
def mapping_columns(df):
    # Replace values in the 'accident' column
    df["accident"] = df["accident"].replace({
        'At least 1 accident or damage reported': 1,
        'None reported': 0
    })
    
    # Replace values in the 'clean_title' column
    df["clean_title"] = df["clean_title"].replace({
        "Yes": 1,
        "No": 0
    })
    
    # Replace values in the 'transmission' column
    df["transmission"] = df["transmission"].replace({'Automatic':1,
                                                'Manual':2,
                                                'Variator':3,
                                                'Other':4})

    return df

train_df = mapping_columns(train_df)
test_df = mapping_columns(test_df)


## Handling Categorical Value


In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, ElasticNet

# Encoding categorical variables
categorical_columns = ['brand', 'fuel_type', 'ext_col', 'int_col']
lb = LabelEncoder()

for col in categorical_columns:
    if col in train_df.columns:
        train_df[col] = lb.fit_transform(train_df[col])
        test_df[col] = lb.transform(test_df[col])



In [ ]:
train_df.reset_index(drop=True, inplace=True)

In [ ]:


plt.figure(figsize=(12, 8))
sns.heatmap(train_df.corr(), annot=True, cmap='inferno', fmt='.2f')
plt.title('Correlation Heatmap')
plt.show()



In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(train_df['price'], kde=True, color='blue')
plt.title('Price Distribution')
plt.xlabel('Price')
plt.show()

In [ ]:


# Log transform the highly skewed target variable (price)
train_df['price'] = np.log1p(train_df['price'])



In [ ]:


plt.figure(figsize=(10, 6))
sns.histplot(train_df['price'], kde=True, color='blue')
plt.title('Price Distribution')
plt.xlabel('Price')
plt.show()




In [ ]:

# Set the number of rows and columns for the grid
n_rows = 4
n_cols = 4

# Filter numerical columns only
num_columns = train_df.select_dtypes(include=['float64', 'int64']).columns

# Create the figure and axes
fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 12))

# Flatten axes array to easily iterate over
axes = axes.flatten()

# Plot each numeric column in a subplot
for i, col in enumerate(num_columns):
    sns.histplot(train_df[col], kde=True, ax=axes[i], color='blue')
    axes[i].set_title(f'Distribution of {col}', fontsize=12)

# If there are more subplots than features, remove the extra subplots
for j in range(len(num_columns), len(axes)):
    fig.delaxes(axes[j])

# Adjust layout
plt.tight_layout()


## Splitting Datas


In [ ]:
X = train_df.drop(['price'],axis=1)
y = train_df['price']
X.shape, y.shape


In [ ]:
print(df.columns)

## Scaling datas


In [ ]:
scaler = StandardScaler()

# Scale continuous features
continuous_features = ['milage', 'Horsepower', 'Displacement', 'Cylinder Count', 'model_age']
train_df[continuous_features] = scaler.fit_transform(train_df[continuous_features])

# Scale the same way for the test set
test_df[continuous_features] = scaler.transform(test_df[continuous_features])

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


## Model Building # Linear Regression

In [ ]:
# Assuming 'df' is your DataFrame
# First, let's see what columns are available
print(df.columns)

# Identify your target column (let's say it's 'price' for this example)
target_column = 'price'  # Replace 'price' with your actual target column name

# Select features (X) and target (y)
numeric_columns = df.select_dtypes(include=[np.number]).columns
X = df[numeric_columns].drop(columns=[target_column], errors='ignore')
y = df[target_column]

# Now split the data and fit the model
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model_lr = LinearRegression()
model_lr.fit(X_train, y_train)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression  # This line is corrected

# Assuming 'df' is your DataFrame
target_column = 'price'

# Select features (X) and target (y)
numeric_columns = df.select_dtypes(include=[np.number]).columns
X = df[numeric_columns].drop(columns=[target_column], errors='ignore')
y = df[target_column]

# Now split the data and fit the model
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model_lr = LinearRegression()
model_lr.fit(X_train, y_train)

# Predict and evaluate
y_pred = model_lr.predict(X_val)
rmse = np.sqrt(np.mean((y_pred - y_val)**2))
print(f'Linear Regression RMSE: {rmse:.2f}')

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Assuming 'df' is your DataFrame
target_column = 'price'

# Separate features and target
X = df.drop(columns=[target_column])
y = df[target_column]

# Identify column types
numeric_features = ['id', 'model_year', 'milage']
categorical_features = ['brand', 'fuel_type', 'transmission', 'ext_col', 'int_col']
binary_features = ['accident', 'clean_title']

# OneHotEncoder for categorical and binary features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_features),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_features),
        ('bin', OneHotEncoder(drop='first', sparse_output=False), binary_features)
    ])

# Create pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the model
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = pipeline.predict(X_val)
rmse = np.sqrt(np.mean((y_pred - y_val)**2))
print(f'Linear Regression RMSE: {rmse:.2f}')


In [ ]:
df = df.head(125690)

In [ ]:
df.to_csv('submission.csv', index=False)

In [1]:
pip freeze > requirements.txt


Note: you may need to restart the kernel to use updated packages.
